<a href="https://colab.research.google.com/github/jkhyjkhy/NLP_Project/blob/main/%EB%8F%99%EC%A0%81_%ED%8E%98%EC%9D%B4%EC%A7%80_%EC%B6%94%EC%B6%9C_%EB%B0%8F_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# colab에서 실습하기 위한 부분이라 신경 쓰지 않으셔도 됩니다.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.c1yUcg37o6/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.k0GhqiZZon/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.UbMo5ZWq87/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

selenium으로 동적 페이지 고정 후 BeautifulSoup 이용

In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver

url1 = "https://sports.news.naver.com/wfootball/news/index?isphoto=N&type=latest&page=1"
url2 = "https://sports.news.naver.com/wfootball/news/index?isphoto=N&type=latest&page=2"
url3 = "https://sports.news.naver.com/wfootball/news/index?isphoto=N&type=latest&page=3"

url_list = [url1, url2, url3]

In [ ]:
import datetime as dt
now = dt.datetime.now()
now_hour = now.hour
need_hour = str(now_hour+8)
print(now_hour)
print(need_hour)

8
16


In [ ]:
newsData = []

In [ ]:

for i in range(0, len(url_list)):
  url = url_list[i]
  driver.get(url)
  soup = bs(driver.page_source,'html.parser')
  newslist = soup.select("#_newsList.news_list > ul")

  for news in newslist:
    lis = news.findAll("li")
    for li in lis:
        list_time = li.select_one('.source > .time')
        news_time = list_time.text
        news_hour = news_time[11:13]

        if need_hour == news_hour:
          list_title = li.select_one(".text > a > span")
          news_title = list_title.text
          a = li.select_one(".text > a")
          news_link = 'https://sports.naver.com/' + a.attrs['href']
          newsData.append({
            'title': news_title,
            'link': news_link,
            'time': news_time,
          })

[<ul>
<li>
<a class="thmb" href="/wfootball/news/read?oid=139&amp;aid=0002193472" onclick="clickcr(this, 'nwl.image', '', '', event);">
<img alt="어떻게든 지킨다! 맨시티, 홀란드와 재계약 협상 추진" class="lazyLoadImage" lazy-src="https://imgnews.pstatic.net/image/origin/139/2023/11/20/2193472.jpg?type=w140" src="https://imgnews.pstatic.net/image/origin/139/2023/11/20/2193472.jpg?type=w140" width="140"/>
<span class="mask"></span>
</a>
<div class="text">
<a class="title" href="/wfootball/news/read?oid=139&amp;aid=0002193472" onclick="clickcr(this, 'nwl.title', '', '', event);"><span>어떻게든 지킨다! 맨시티, 홀란드와 재계약 협상 추진</span></a>
<p class="desc">[스포탈코리아] 남정훈 기자= 맨시티가 홀란드는 무조건 지키려고 한다.영국 매체 ‘90min’은 19일(한국 시간) “맨체스터 시티가 엘링 홀란드의 대리인과...</p>
<div class="source">
<span class="press">스포탈코리아</span>
<span class="time"><span class="bar"></span>2023.11.20 17:06</span>
</div>
</div>
</li>
<li>
<a class="thmb" href="/wfootball/news/read?oid=109&amp;aid=0004970368" onclick="clickcr(this, 'nwl.image', '', '', event);">
<img al

In [ ]:
import numpy as np

print('-----------------------------')
newsData

-----------------------------


[{'title': '황희찬, 울버햄프턴과 연장 계약 논의…4800만 원 주급, 얼마나 뛸까?',
  'link': 'https://sports.naver.com//wfootball/news/read?oid=382&aid=0001088831',
  'time': '2023.11.20 16:57'},
 {'title': "히샬리송도 나오는데 난 왜 못 나와!...토트넘 '탈출' 모색",
  'link': 'https://sports.naver.com//wfootball/news/read?oid=411&aid=0000037562',
  'time': '2023.11.20 16:56'},
 {'title': 'LEE 동료의 위엄…‘메날두’보다 빠르게 300호 골 고지 점령',
  'link': 'https://sports.naver.com//wfootball/news/read?oid=241&aid=0003314023',
  'time': '2023.11.20 16:55'},
 {'title': "'EPL에서 증명할래!' 히샬리송, 사우디 이적 대신 토트넘 잔류 유력",
  'link': 'https://sports.naver.com//wfootball/news/read?oid=139&aid=0002193470',
  'time': '2023.11.20 16:52'},
 {'title': 'KIM 따라 바이에른 뮌헨 갈까...“거함들이 관심 보인다”',
  'link': 'https://sports.naver.com//wfootball/news/read?oid=216&aid=0000129532',
  'time': '2023.11.20 16:52'},
 {'title': "'야인' 데 헤아, 맨유 선배가 돕는다…'은퇴하지 말고 뛰자'",
  'link': 'https://sports.naver.com//wfootball/news/read?oid=413&aid=0000168712',
  'time': '2023.11.20 16:51'},
 {'title': "'십자인대

키워드 추출 및 추천 모델

In [ ]:
!pip install keybert
!pip install kiwipiepy
!pip install transformers


In [ ]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel



In [ ]:
model = BertModel.from_pretrained('skt/kobert-base-v1') # 사전 학습된 지도학습 자연어처리 언어 모델 호출(AI 모델 선언)
kw_model = KeyBERT(model) # 키워드 추출 모델 선언, skt/kobert-base-v1 사용

kiwi = Kiwi() # 형태소 분석기 모델 선언

In [ ]:
list_a = []
for news in newsData:
  text = news['title'] # 뉴스에서 제목만을 가지고 키워드 추출해보기
  for word in kiwi.analyze(text): # 키위 분석기에
    nouns = [token.form for token in word[0] if token.tag.startswith('NN')]
  result_text = ' '.join(nouns)
  keywords = kw_model.extract_keywords(result_text, keyphrase_ngram_range=(1, 2), stop_words = None, top_n=5)
  list_a.append(keywords)
  print(keywords)

[('황희찬 울버햄프턴', 0.8297), ('울버햄프턴', 0.7823), ('계약 논의', 0.7218), ('울버햄프턴 연장', 0.7099), ('계약', 0.7033)]
[('히샬리송 토트넘', 0.9913), ('히샬리송', 0.9182), ('토트넘', 0.6092), ('토트넘 탈출', 0.5887), ('모색', 0.5769)]
[('위엄 메날두', 0.952), ('메날두 고지', 0.9343), ('메날두', 0.8879), ('고지 점령', 0.8055), ('고지', 0.7934)]
[('대신 토트넘', 0.8945), ('히샬리송 사우디', 0.89), ('히샬리송', 0.8388), ('사우디 이적', 0.7722), ('사우디', 0.7108)]
[('뮌헨 거함', 0.9634), ('바이에른 뮌헨', 0.9426), ('뮌헨', 0.8777), ('바이에른', 0.8021), ('거함', 0.6825)]
[('맨유 선배', 0.8147), ('헤아 맨유', 0.7925), ('야인 헤아', 0.7883), ('야인', 0.7666), ('헤아', 0.7233)]
[('부상 의심', 0.6401), ('십자 인대', 0.5769), ('바르사 출장', 0.5739), ('자체 분노', 0.5566), ('십자', 0.5476)]
[('영입 햄스트링', 0.7999), ('조기 복귀', 0.7455), ('조기', 0.6975), ('햄스트링 수술', 0.6866), ('수술 조기', 0.6849)]
[('선정 토트넘', 0.7589), ('손흥민 영국', 0.743), ('토트넘', 0.7095), ('토트넘 역사', 0.6751), ('영국', 0.6283)]
[('시티 트레블', 0.8713), ('트레블 주역', 0.8451), ('시티', 0.7892), ('주역', 0.766), ('주역 결심', 0.6341)]
[('대회 월드컵', 0.8979), ('월드컵 이면', 0.8783), ('월드컵', 0.8597), ('세계

이중 그럼 이슈가 되는 키워드는 어떻게?(가장 간단한 빈도분석 방법으로 처리)

In [ ]:
from collections import Counter

def get_top_keywords(documents_keywords):
    # 각 문서의 상위 키워드들을 담을 리스트
    all_keywords = []

    # 각 문서에서 추출된 상위 5개 키워드를 하나의 리스트로 합치기
    for document in documents_keywords:
        top_keywords = [keyword[0] for keyword in document]
        all_keywords.extend(top_keywords)

    # 공백을 기준으로 단어를 분리하여 중복을 제거하고 키워드의 빈도 계산
    keyword_counter = Counter(keyword for keywords in all_keywords for keyword in keywords.split())

    # 빈도수를 기준으로 상위 6개 키워드 추출
    top_6_keywords = keyword_counter.most_common(6)

    return top_6_keywords


In [ ]:
print(list_a)

[[('황희찬 울버햄프턴', 0.8297), ('울버햄프턴', 0.7823), ('계약 논의', 0.7218), ('울버햄프턴 연장', 0.7099), ('계약', 0.7033)], [('히샬리송 토트넘', 0.9913), ('히샬리송', 0.9182), ('토트넘', 0.6092), ('토트넘 탈출', 0.5887), ('모색', 0.5769)], [('위엄 메날두', 0.952), ('메날두 고지', 0.9343), ('메날두', 0.8879), ('고지 점령', 0.8055), ('고지', 0.7934)], [('대신 토트넘', 0.8945), ('히샬리송 사우디', 0.89), ('히샬리송', 0.8388), ('사우디 이적', 0.7722), ('사우디', 0.7108)], [('뮌헨 거함', 0.9634), ('바이에른 뮌헨', 0.9426), ('뮌헨', 0.8777), ('바이에른', 0.8021), ('거함', 0.6825)], [('맨유 선배', 0.8147), ('헤아 맨유', 0.7925), ('야인 헤아', 0.7883), ('야인', 0.7666), ('헤아', 0.7233)], [('부상 의심', 0.6401), ('십자 인대', 0.5769), ('바르사 출장', 0.5739), ('자체 분노', 0.5566), ('십자', 0.5476)], [('영입 햄스트링', 0.7999), ('조기 복귀', 0.7455), ('조기', 0.6975), ('햄스트링 수술', 0.6866), ('수술 조기', 0.6849)], [('선정 토트넘', 0.7589), ('손흥민 영국', 0.743), ('토트넘', 0.7095), ('토트넘 역사', 0.6751), ('영국', 0.6283)], [('시티 트레블', 0.8713), ('트레블 주역', 0.8451), ('시티', 0.7892), ('주역', 0.766), ('주역 결심', 0.6341)], [('대회 월드컵', 0.8979), ('월드컵 이면', 0.8783), ('월드컵', 0.

In [ ]:
# 전체 문서에서 상위 10개 키워드 추출
#
top_6_keywords = get_top_keywords(list_a)
print(top_6_keywords)

[('토트넘', 10), ('뮌헨', 8), ('히샬리송', 7), ('바이에른', 4), ('바르셀로나', 4), ('울버햄프턴', 3)]
